In [1]:
%%capture
! pip install -qU langchain-google-community \
                  google-cloud-discoveryengine \
                  langchain_ollama \
                  langchain_openai \
                  langchain_community

In [42]:
from langchain_google_community import VertexAIMultiTurnSearchRetriever
from langchain_google_community import VertexAISearchRetriever
from langchain.schema.output_parser import StrOutputParser
from langchain_ollama import ChatOllama
from langchain_ollama.llms import OllamaLLM
from langchain.llms import Ollama
from langchain.schema.runnable import RunnablePassthrough
from langchain_core.prompts import PromptTemplate

PROJECT_ID = "gsd-ai-mx-ferneutron"
LOCATION = "global"
SEARCH_ENGINE_ID = "llm-demo-1"  # Set to your search app ID
DATA_STORE_ID = "llms-demo-1_1728585032210"  # Set to your data store ID

In [34]:
retriever = VertexAISearchRetriever(
    project_id=PROJECT_ID,
    location_id=LOCATION,
    data_store_id=DATA_STORE_ID,
    max_documents=1,
)


In [45]:
%%time
query = "what about llms"

result = retriever.invoke(query)
for doc in result:
    print(doc)
    break

page_content='tens to hundreds of billions of parameters, which are pre
trained on massive text data, such as PaLM [31], LLaMA
[32], and GPT-4 [33], as summarized in Table III. Compared
1Recently, several very promising non-transformer LLMs have been pro
posed, such as the LLMs based on structured state space models [29], [30].
See Section VII for more details.

arXiv:2402.06196v2 [cs.CL] 20 Feb 2024' metadata={'id': '3d57d97009ba640efd886145022e0071', 'source': 'gs://demo-datastore-llms/llm-page-1.pdf1', 'previous_segments': [{'content': 'that have different starting points and velocity: statistical lan\nguage models, neural language models, pre-trained language\nmodels and LLMs.\nStatistical language models (SLMs) view text as a sequence\nof words, and estimate the probability of text as the product\nof their word probabilities. The dominating form of SLMs\nare Markov chain models known as the n-gram models,\nwhich compute the probability of a word conditioned on its\nimmediate proce

In [35]:
prompt = PromptTemplate(
    input_variables=["context", "question"],
    template="""Use the following context to answer the question at the end. 
                If you don't know the answer, just say that you don't know, don't try   
                to make up an answer.\n\nContext: {context}\n\nQuestion: {question}"""
)

model = OllamaLLM(model="llama3.1")
# model = ChatOllama(model="llama3.1")

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
   # | StrOutputParser()
)

In [39]:
%%time
x = chain.invoke("what abou llms?")

CPU times: user 200 ms, sys: 47.7 ms, total: 248 ms
Wall time: 52.9 s


In [40]:
x

"The context discusses Large Language Models (LLMs) extensively, but I'll try to summarize the key points related to them:\n\n* LLMs are a type of model that has drawn significant attention due to their strong performance on natural language tasks.\n* They have been pre-trained on massive amounts of text data, which enables general-purpose language understanding and generation.\n* The research area of LLMs is evolving rapidly and is discussed in the paper as one of the four waves of language modeling.\n* Three popular LLM families (GPT, LLaMA, PaLM) are reviewed in the paper, along with their characteristics, contributions, and limitations.\n* LLMs have become a basic building block for developing general-purpose AI agents or artificial general intelligence (AGI).\n* The field of LLMs is moving fast, with new findings, models, and techniques being published rapidly.\n\nHowever, I don't see an explicit mention of what happens to LLMs in the provided context. Could you please provide mor

In [38]:
x = chain.invoke("what is the summary of llms?")
print(x)

Based on the provided context, here is a summary of LLMS (Large Language Models):

LLMs are pre-trained statistical language models based on neural networks. They have been successful due to decades of research and development in language modeling, which can be categorized into four waves:

1. Statistical Language Models (SLMs)
2. Early Neural Language Models (NLMs)
3. Pre-Trained Language Models (PLMs)
4. Large Language Models (LLMs)

LLMs are transformer-based models pre-trained on Web-scale text corpora and have become the basic building block for general-purpose AI agents or artificial general intelligence (AGI). They can be used as task solvers, power systems like Microsoft's Co-Pilot, and perform multi-step reasoning.

Key characteristics of LLMs include:

* Pre-training on large amounts of text data
* Ability to handle natural language understanding and generation tasks
* Capacity for general-purpose language understanding and generation
* Use of neural networks and transformer-

In [43]:

from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.chains import RetrievalQA

# Prompt
template = """Use the following pieces of context to answer the question at the end.
If you don't know the answer, just say that you don't know, don't try to make up an answer.
Use three sentences maximum and keep the answer as concise as possible.
{context}
Question: {question}
Helpful Answer:"""

QA_CHAIN_PROMPT = PromptTemplate(
    input_variables=["context", "question"],
    template=template,
)

llm = Ollama(model="llama3.1", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
qa_chain = RetrievalQA.from_chain_type(
    llm,
    retriever=retriever,
    chain_type_kwargs={"prompt": QA_CHAIN_PROMPT},
    )

/var/tmp/ipykernel_4255/408293162.py:18: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="llama3.1", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))
/var/tmp/ipykernel_4255/408293162.py:18: DeprecationWarning: callback_manager is deprecated. Please use callbacks instead.
  llm = Ollama(model="llama3.1", callback_manager=CallbackManager([StreamingStdOutCallbackHandler()]))


In [44]:
%%time
result = qa_chain({"query": "what about llms?"})

<timed exec>:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.


Recently, several non-transformer LLMs have been proposed, such as those based on structured state space models. These alternative approaches aim to improve upon traditional transformer-based LLMs like PaLM and GPT-4.CPU times: user 180 ms, sys: 19.4 ms, total: 199 ms
Wall time: 34.5 s
